In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    %pip uninstall -y plotnine
    !git clone https://github.com/bzhanglab/CoPheeMap.git
    %cd CoPheeMap
    %pip install -q -r requirements.txt

In [ ]:
import pandas as pd
import re
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sts
import math
from sklearn import metrics
from statannot import add_stat_annotation
from collections import Counter
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from os import listdir
from os.path import isfile
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import networkx as nx
import pickle
import joblib
import xgboost as xgb
import logging

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [ ]:
Cancer=['BRCA','CCRCC','COAD','GBM','HCC','HNSCC','LSCC','LUAD','OV','PDAC','UCEC'] 

In [ ]:
#Kinase name, family, group information
kinases_id=pd.read_excel('KSA/kinase_table.xlsx')
kinases_id.index=kinases_id.Name

#gold standard KSAs
K_S_GPS=pd.read_csv('KSA/KSA_gold_standard.csv',index_col=0)
ks=list(set(K_S_GPS['Gene name']))

#All sites identified in the CPTAC PanCan datasets
a_file = open("PanCan/all_sites_id_1cancer.pkl", "rb")
All_sites = pickle.load(a_file)

In [ ]:
#Classify kinases into under-studied and other kinases according to # of known substrates
poor_kinase=[]
other_kinase=[]
poor_num=[]
other_num=[]
for i in ks:
    if 'family' not in i:
        if 'Isoform' not in i:
            tmp=K_S_GPS[K_S_GPS['Gene name']==i]
            if len(set(tmp['seq_15']))<=10:
                poor_kinase.append(i)
                poor_num.append(len(set(tmp['seq_15'])))
            if len(set(tmp['seq_15']))>10:
                other_kinase.append(i)
                other_num.append(len(set(tmp['seq_15'])))

In [ ]:
#Load CoPheeMap
network = pd.read_csv('Supplementary_table/Table_S2_CoPheeMap.tsv.zip', compression='zip', sep='\t')

seq1=[i.split('|')[3] for i in network.site1.tolist()]
seq2=[i.split('|')[3] for i in network.site2.tolist()]
network['seq1']=np.array(seq1)
network['seq2']=np.array(seq2)
sites=list(set(network.site1.tolist()+network.site2.tolist()))
G = nx.Graph()
elist=list(zip(network.site1.tolist(),network.site2.tolist()))
G.add_edges_from(elist)

idx_ST=[]
idx_Y=[]
for i in range(len(network)):
    if network.seq1.iloc[i][7] in ['S','T']:
        idx_ST.append(i)
    if network.seq1.iloc[i][7] in ['Y']:
        idx_Y.append(i)
network_ST=network.iloc[idx_ST,:]
network_Y=network.iloc[idx_Y,:]
lst_ST=list(set(network_ST.site1.tolist()+network_ST.site2.tolist()))
H=G.subgraph(lst_ST)
nodes_ST=max(nx.connected_components(H))
G_ST=G.subgraph(nodes_ST)
network_Y=network.iloc[idx_Y,:]
lst_Y=list(set(network_Y.site1.tolist()+network_Y.site2.tolist()))
H=G.subgraph(lst_Y)
nodes_Y=max(nx.connected_components(H))
G_Y=G.subgraph(nodes_Y)

In [ ]:
#Load embeddings of KMap and CoPheeMap
kk_emb=pd.read_csv('CoPheeKSA/kinase_network/n2v_KK_PPI_PCC.csv',index_col=0)
cophee_emb=pd.read_csv('CoPheeMap/data_construction/n2v_networkST.csv', index_col=0)

In [ ]:
#Supervised model trained with one static feature: PSSM scores
auc=[]
label=[]
for i in range(10):
    data_train=pd.read_csv('CoPheeKSA/static_features/test/test_'+str(i)+'.csv',index_col=0)
    data_train=data_train.drop_duplicates()
    data_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    data_train=data_train.dropna()
    data_neg=pd.read_csv('Supplementary_table/Table_S3_negative_KSA_w_features.tsv.zip', compression='zip', sep='\t')
    data_neg=data_neg.drop_duplicates()
    idx=random.sample(range(len(data_neg)),10*len(data_train))
    data_neg=data_neg.iloc[idx]
    
    y=np.concatenate((np.repeat(1,len(data_train)),np.repeat(0,len(data_neg))))
    y_pred=np.concatenate((data_train.PSSM_score,data_neg.PSSM))
    fpr, tpr, _ = metrics.roc_curve(y,  y_pred)
    auc.append(metrics.auc(fpr, tpr))
    label.append('PSSM')

In [ ]:
#Supervised model trained with dynamic features + PSSM scores + CoPheeMap info + KMap info
Cancer=['BRCA','CCRCC','COAD','GBM','HCC','HNSCC','LSCC','LUAD','OV','PDAC','UCEC'] 
col_idx=['cophee_emb0', 'cophee_emb1', 'cophee_emb2', 'cophee_emb3', 'cophee_emb4', 
         'cophee_emb5', 'cophee_emb6', 'cophee_emb7', 'cophee_emb8', 'cophee_emb9', 
         'cophee_emb10','cophee_emb11', 'cophee_emb12', 'cophee_emb13', 'cophee_emb14', 
         'cophee_emb15', 'k_emb0', 'k_emb1', 'k_emb2', 'k_emb3','k_emb4', 'k_emb5', 
         'k_emb6', 'k_emb7', 'k_emb8', 'k_emb9', 'k_emb10',
         'k_emb11', 'k_emb12', 'k_emb13', 'k_emb14', 'k_emb15','BRCA_spc_k_s', 
         'CCRCC_spc_k_s', 'COAD_spc_k_s', 'GBM_spc_k_s',
         'HCC_spc_k_s', 'HNSCC_spc_k_s', 'LSCC_spc_k_s', 'LUAD_spc_k_s',
         'OV_spc_k_s', 'PDAC_spc_k_s', 'UCEC_spc_k_s', 'BRCA_spc_kactivity_s',
         'CCRCC_spc_kactivity_s', 'COAD_spc_kactivity_s', 'GBM_spc_kactivity_s',
         'HCC_spc_kactivity_s', 'HNSCC_spc_kactivity_s', 'LSCC_spc_kactivity_s',
         'LUAD_spc_kactivity_s', 'OV_spc_kactivity_s', 'PDAC_spc_kactivity_s',
         'UCEC_spc_kactivity_s','PSSM']
for i in range(10):
    data_train=pd.read_csv('CoPheeKSA/train/w_feature/train_'+str(i)+'.csv',index_col=0)
    data_train=data_train.drop_duplicates()
    data_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    data_train=data_train[~data_train['PSSM'].isna()]
    
    data_neg=pd.read_csv('Supplementary_table/Table_S3_negative_KSA_w_features.tsv.zip', compression='zip', sep='\t')
    data_neg=data_neg.drop_duplicates()
    data_neg.index=range(len(data_neg))
    
    kinases=list(set(data_train.kinase))
    data_train_neg=pd.DataFrame()
    for k in kinases:
        tmp=data_train[data_train.kinase==k]
        tmp2=data_neg[data_neg.kinase==k]
        if len(tmp2) > 10*len(tmp):
            idx=random.sample(range(len(tmp2)),10*len(tmp))
            tmp2=tmp2.iloc[idx]
        
        c=tmp[['BRCA_spc_kactivity_s',
             'CCRCC_spc_kactivity_s', 'COAD_spc_kactivity_s', 'GBM_spc_kactivity_s',
             'HCC_spc_kactivity_s', 'HNSCC_spc_kactivity_s', 'LSCC_spc_kactivity_s',
             'LUAD_spc_kactivity_s', 'OV_spc_kactivity_s', 'PDAC_spc_kactivity_s',
             'UCEC_spc_kactivity_s']].isna().sum()
        for idx_col in ['BRCA_spc_kactivity_s',
             'CCRCC_spc_kactivity_s', 'COAD_spc_kactivity_s', 'GBM_spc_kactivity_s',
             'HCC_spc_kactivity_s', 'HNSCC_spc_kactivity_s', 'LSCC_spc_kactivity_s',
             'LUAD_spc_kactivity_s', 'OV_spc_kactivity_s', 'PDAC_spc_kactivity_s',
             'UCEC_spc_kactivity_s']:
            num=int(c.loc[idx_col]/len(tmp)*len(tmp2))
            idx=random.sample(tmp2.index.tolist(),num)
            
            for z in idx:
                tmp2.loc[z, idx_col]=np.nan
            
        data_train_neg=pd.concat([data_train_neg,tmp2])
    idx=random.sample(range(len(data_neg)),2*len(data_train))
    data_train_neg=pd.concat([data_train_neg,data_neg.iloc[idx]])
    data_train_neg=data_train_neg.drop_duplicates()
    
    idx_removed=data_train_neg.index
    data_neg=data_neg[~data_neg.index.isin(idx_removed)]
    data_neg.index=range(len(data_neg))
    
    data_test=pd.read_csv('CoPheeKSA/test/w_feature/test_'+str(i)+'.csv',index_col=0)
    data_test=data_test.drop_duplicates()
    data_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    data_test=data_test[~data_test['PSSM'].isna()]
    
    idx=random.sample(range(len(data_test)),25)
    data_eval=data_test.iloc[idx,:]
    idx2=[i for i in range(len(data_test)) if i not in idx]
    data_test=data_test.iloc[idx2,:]
    
    idx=random.sample(range(len(data_neg)),10*len(data_eval))
    data_eval_neg=data_neg.iloc[idx]
    idx_removed=data_eval_neg.index
    data_neg=data_neg[~data_neg.index.isin(idx_removed)]
    data_neg.index=range(len(data_neg))
    
    idx=random.sample(range(len(data_neg)),10*len(data_test))
    data_test_neg=data_neg.iloc[idx]
    idx_removed=data_test_neg.index
    data_neg=data_neg[~data_neg.index.isin(idx_removed)]
    
    data_train=data_train[col_idx]
    data_eval=data_eval[col_idx]
    data_test=data_test[col_idx]
    data_neg=data_neg[col_idx]
    data_train_neg=data_train_neg[col_idx]
    data_eval_neg=data_eval_neg[col_idx]
    
    X_train=pd.concat([data_train,data_train_neg])
    y_train=np.concatenate((np.repeat(1,len(data_train)),np.repeat(0,len(data_train_neg))))
    X_train=X_train[col_idx]
    X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    
    X_eval=pd.concat([data_eval,data_eval_neg])
    y_eval=np.concatenate((np.repeat(1,len(data_eval)),np.repeat(0,len(data_eval_neg))))
    X_eval=X_eval[col_idx]
    X_eval.replace([np.inf, -np.inf], np.nan, inplace=True)
    deval = xgb.DMatrix(X_eval,label=y_eval)
    param = {'max_depth': 2, 'eta': 0.2, 'objective': 'binary:logistic','verbosity':0}
    param['nthread'] = 4
    param['eval_metric'] = 'auc'
    evallist = [(dtrain, 'train'),(deval, 'eval')]
    num_round = 300
    bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=10)
    
    idx=random.sample(range(len(data_neg)),10*len(data_test))
    data_neg2=data_neg.iloc[idx,:]
    X_test=pd.concat([data_test,data_neg2])
    X_test=X_test[col_idx]
    X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    y_test=np.concatenate((np.repeat(1,len(data_test)),np.repeat(0,len(data_neg2))))
    dtest = xgb.DMatrix(X_test, label=y_test)
    y_pred = bst.predict(dtest)

    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred)
    auc_tmp=metrics.auc(fpr, tpr)
    
    auc.append(auc_tmp)
    label.append('CoPheeKSA')

In [ ]:
#Supervised model trained with CoPheeMap info + KMap info
Cancer=['BRCA','CCRCC','COAD','GBM','HCC','HNSCC','LSCC','LUAD','OV','PDAC','UCEC'] 
col_idx=['cophee_emb0', 'cophee_emb1', 'cophee_emb2', 'cophee_emb3', 'cophee_emb4', 
         'cophee_emb5', 'cophee_emb6', 'cophee_emb7', 'cophee_emb8', 'cophee_emb9', 
         'cophee_emb10','cophee_emb11', 'cophee_emb12', 'cophee_emb13', 'cophee_emb14', 
         'cophee_emb15', 'k_emb0', 'k_emb1', 'k_emb2', 'k_emb3','k_emb4', 'k_emb5', 
         'k_emb6', 'k_emb7', 'k_emb8', 'k_emb9', 'k_emb10',
         'k_emb11', 'k_emb12', 'k_emb13', 'k_emb14', 'k_emb15']
for i in range(10):
    data_neg=pd.read_csv('Supplementary_table/Table_S3_negative_KSA_w_features.tsv.zip', compression='zip', sep='\t')
    data_neg=data_neg.drop_duplicates()
    data_neg=data_neg.replace(0,np.nan)
    data_neg.index=range(len(data_neg))
    #data_neg=data_neg[~data_neg['PSSM'].isna()]
    
    data_train=pd.read_csv('CoPheeKSA/train/w_feature/train_'+str(i)+'.csv',index_col=0)
    data_train=data_train.drop_duplicates()
    data_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    #data_train=data_train[~data_train['PSSM'].isna()]
    
    kinases=list(set(data_train.kinase))
    data_train_neg=pd.DataFrame()
    for k in kinases:
        tmp=data_train[data_train.kinase==k]
        tmp2=data_neg[data_neg.kinase==k]
        if len(tmp2) > 10*len(tmp):
            idx=random.sample(range(len(tmp2)),10*len(tmp))
            tmp2=tmp2.iloc[idx]
            
        data_train_neg=pd.concat([data_train_neg,tmp2])
    idx=random.sample(range(len(data_neg)),2*len(data_train))
    data_train_neg=pd.concat([data_train_neg,data_neg.iloc[idx]])
    data_train_neg=data_train_neg.drop_duplicates()
    
    idx_removed=data_train_neg.index
    data_neg=data_neg[~data_neg.index.isin(idx_removed)]
    data_neg.index=range(len(data_neg))
    
    data_test=pd.read_csv('CoPheeKSA/test/w_feature/test_'+str(i)+'.csv',index_col=0)
    data_test=data_test.drop_duplicates()
    data_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    #data_test=data_test[~data_test['PSSM'].isna()]
    
    idx=random.sample(range(len(data_test)),25)
    data_eval=data_test.iloc[idx,:]
    idx2=[i for i in range(len(data_test)) if i not in idx]
    data_test=data_test.iloc[idx2,:]
    
    idx=random.sample(range(len(data_neg)),10*len(data_eval))
    data_neg_eval=data_neg.iloc[idx,:]
    idx2=[i for i in range(len(data_neg)) if i not in idx]
    data_neg=data_neg.iloc[idx2,:]
    
    
    data_train=data_train[col_idx]
    data_eval=data_eval[col_idx]
    data_test=data_test[col_idx]
    data_neg=data_neg[col_idx]
    data_train_neg=data_train_neg[col_idx]
    data_neg_eval=data_neg_eval[col_idx]
    
    X_train=pd.concat([data_train,data_train_neg])
    y_train=np.concatenate((np.repeat(1,len(data_train)),np.repeat(0,len(data_train_neg))))
    X_train=X_train[col_idx]
    X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    #X_train=X_train.replace(np.nan,0)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    
    X_eval=pd.concat([data_eval,data_neg_eval])
    y_eval=np.concatenate((np.repeat(1,len(data_eval)),np.repeat(0,len(data_neg_eval))))
    X_eval=X_eval[col_idx]
    X_eval.replace([np.inf, -np.inf], np.nan, inplace=True)
    deval = xgb.DMatrix(X_eval,label=y_eval)
    param = {'max_depth': 2, 'eta': 0.2, 'objective': 'binary:logistic','verbosity':0}
    param['nthread'] = 4
    param['eval_metric'] = 'auc'
    evallist = [(dtrain, 'train'),(deval, 'eval')]
    num_round = 300
    bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=10)
    
    idx=random.sample(range(len(data_neg)),10*len(data_test))
    data_neg2=data_neg.iloc[idx,:]
    X_test=pd.concat([data_test,data_neg2])
    X_test=X_test[col_idx]
    X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    y_test=np.concatenate((np.repeat(1,len(data_test)),np.repeat(0,len(data_neg2))))
    dtest = xgb.DMatrix(X_test, label=y_test)
    y_pred = bst.predict(dtest)

    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred)
    auc_tmp=metrics.auc(fpr, tpr)
    logging.info(auc_tmp)
    
    auc.append(auc_tmp)
    label.append('CoPheeMap+KMap')

In [ ]:
#Supervised model trained with PSSM scores + CoPheeMap info + KMap info
Cancer=['BRCA','CCRCC','COAD','GBM','HCC','HNSCC','LSCC','LUAD','OV','PDAC','UCEC'] 
col_idx=['cophee_emb0', 'cophee_emb1', 'cophee_emb2', 'cophee_emb3', 'cophee_emb4', 
         'cophee_emb5', 'cophee_emb6', 'cophee_emb7', 'cophee_emb8', 'cophee_emb9', 
         'cophee_emb10','cophee_emb11', 'cophee_emb12', 'cophee_emb13', 'cophee_emb14', 
         'cophee_emb15', 'k_emb0', 'k_emb1', 'k_emb2', 'k_emb3','k_emb4', 'k_emb5', 
         'k_emb6', 'k_emb7', 'k_emb8', 'k_emb9', 'k_emb10',
         'k_emb11', 'k_emb12', 'k_emb13', 'k_emb14', 'k_emb15','PSSM']
for i in range(10):
    data_neg=pd.read_csv('Supplementary_table/Table_S3_negative_KSA_w_features.tsv.zip', compression='zip', sep='\t')
    data_neg=data_neg.drop_duplicates()
    data_neg=data_neg.replace(0,np.nan)
    data_neg=data_neg[~data_neg['PSSM'].isna()]
    data_neg.index=range(len(data_neg))
    
    data_train=pd.read_csv('CoPheeKSA/train/w_feature/train_'+str(i)+'.csv',index_col=0)
    data_train=data_train.drop_duplicates()
    data_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    data_train=data_train[~data_train['PSSM'].isna()]
    
    kinases=list(set(data_train.kinase))
    data_train_neg=pd.DataFrame()
    for k in kinases:
        tmp=data_train[data_train.kinase==k]
        tmp2=data_neg[data_neg.kinase==k]
        if len(tmp2) > 10*len(tmp):
            idx=random.sample(range(len(tmp2)),10*len(tmp))
            tmp2=tmp2.iloc[idx]
            
        data_train_neg=pd.concat([data_train_neg,tmp2])
    idx=random.sample(range(len(data_neg)),2*len(data_train))
    data_train_neg=pd.concat([data_train_neg,data_neg.iloc[idx]])
    data_train_neg=data_train_neg.drop_duplicates()
    
    idx_removed=data_train_neg.index
    data_neg=data_neg[~data_neg.index.isin(idx_removed)]
    data_neg.index=range(len(data_neg))
    
    data_test=pd.read_csv('CoPheeKSA/test/w_feature/test_'+str(i)+'.csv',index_col=0)
    data_test=data_test.drop_duplicates()
    data_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    data_test=data_test[~data_test['PSSM'].isna()]
    
    idx=random.sample(range(len(data_test)),25)
    data_eval=data_test.iloc[idx,:]
    idx2=[i for i in range(len(data_test)) if i not in idx]
    data_test=data_test.iloc[idx2,:]
    
    idx=random.sample(range(len(data_neg)),10*len(data_eval))
    data_neg_eval=data_neg.iloc[idx,:]
    idx2=[i for i in range(len(data_neg)) if i not in idx]
    data_neg=data_neg.iloc[idx2,:]
    
    
    data_train=data_train[col_idx]
    data_eval=data_eval[col_idx]
    data_test=data_test[col_idx]
    data_neg=data_neg[col_idx]
    data_neg_eval=data_neg_eval[col_idx]
    data_train_neg=data_train_neg[col_idx]
    
    X_train=pd.concat([data_train,data_train_neg])
    y_train=np.concatenate((np.repeat(1,len(data_train)),np.repeat(0,len(data_train_neg))))
    X_train=X_train[col_idx]
    X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    #X_train=X_train.replace(np.nan,0)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    
    X_eval=pd.concat([data_eval,data_neg_eval])
    y_eval=np.concatenate((np.repeat(1,len(data_eval)),np.repeat(0,len(data_neg_eval))))
    X_eval=X_eval[col_idx]
    X_eval.replace([np.inf, -np.inf], np.nan, inplace=True)
    deval = xgb.DMatrix(X_eval,label=y_eval)
    param = {'max_depth': 2, 'eta': 0.2, 'objective': 'binary:logistic','verbosity':0}
    param['nthread'] = 4
    param['eval_metric'] = 'auc'
    evallist = [(dtrain, 'train'),(deval, 'eval')]
    num_round = 300
    bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=10)
    
    idx=random.sample(range(len(data_neg)),10*len(data_test))
    data_neg2=data_neg.iloc[idx,:]
    X_test=pd.concat([data_test,data_neg2])
    X_test=X_test[col_idx]
    X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    y_test=np.concatenate((np.repeat(1,len(data_test)),np.repeat(0,len(data_neg2))))
    dtest = xgb.DMatrix(X_test, label=y_test)
    y_pred = bst.predict(dtest)

    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred)
    auc_tmp=metrics.auc(fpr, tpr)
    logging.info(auc_tmp)
    
    auc.append(auc_tmp)
    label.append('PSSM+CoPheeMap+KMap')

In [ ]:
#Supervised model trained with dynamic features
Cancer=['BRCA','CCRCC','COAD','GBM','HCC','HNSCC','LSCC','LUAD','OV','PDAC','UCEC'] 
col_idx=['BRCA_spc_k_s', 
         'CCRCC_spc_k_s', 'COAD_spc_k_s', 'GBM_spc_k_s',
         'HCC_spc_k_s', 'HNSCC_spc_k_s', 'LSCC_spc_k_s', 'LUAD_spc_k_s',
         'OV_spc_k_s', 'PDAC_spc_k_s', 'UCEC_spc_k_s', 'BRCA_spc_kactivity_s',
         'CCRCC_spc_kactivity_s', 'COAD_spc_kactivity_s', 'GBM_spc_kactivity_s',
         'HCC_spc_kactivity_s', 'HNSCC_spc_kactivity_s', 'LSCC_spc_kactivity_s',
         'LUAD_spc_kactivity_s', 'OV_spc_kactivity_s', 'PDAC_spc_kactivity_s',
         'UCEC_spc_kactivity_s']
for i in range(10):
    data_neg=pd.read_csv('Supplementary_table/Table_S3_negative_KSA_w_features.tsv.zip', compression='zip', sep='\t')
    data_neg=data_neg.drop_duplicates()
    data_neg=data_neg.replace(0,np.nan)
    data_neg=data_neg[~data_neg['PSSM'].isna()]
    data_neg.index=range(len(data_neg))
    
    data_train=pd.read_csv('CoPheeKSA/train/w_feature/train_'+str(i)+'.csv',index_col=0)
    data_train=data_train.drop_duplicates()
    data_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    #Avoid overfitting caused by missing values
    kinases=list(set(data_train.kinase))
    data_train_neg=pd.DataFrame()
    for k in kinases:
        tmp=data_train[data_train.kinase==k]
        tmp2=data_neg[data_neg.kinase==k]
        if len(tmp2) > 10*len(tmp):
            idx=random.sample(range(len(tmp2)),10*len(tmp))
            tmp2=tmp2.iloc[idx]
        
        c=tmp[['BRCA_spc_kactivity_s',
             'CCRCC_spc_kactivity_s', 'COAD_spc_kactivity_s', 'GBM_spc_kactivity_s',
             'HCC_spc_kactivity_s', 'HNSCC_spc_kactivity_s', 'LSCC_spc_kactivity_s',
             'LUAD_spc_kactivity_s', 'OV_spc_kactivity_s', 'PDAC_spc_kactivity_s',
             'UCEC_spc_kactivity_s']].isna().sum()
        for idx_col in ['BRCA_spc_kactivity_s',
             'CCRCC_spc_kactivity_s', 'COAD_spc_kactivity_s', 'GBM_spc_kactivity_s',
             'HCC_spc_kactivity_s', 'HNSCC_spc_kactivity_s', 'LSCC_spc_kactivity_s',
             'LUAD_spc_kactivity_s', 'OV_spc_kactivity_s', 'PDAC_spc_kactivity_s',
             'UCEC_spc_kactivity_s']:
            num=int(c.loc[idx_col]/len(tmp)*len(tmp2))
            idx=random.sample(tmp2.index.tolist(),num)
            
            for z in idx:
                tmp2.loc[z, idx_col]=np.nan
            
        data_train_neg=pd.concat([data_train_neg,tmp2])
    idx=random.sample(range(len(data_neg)),2*len(data_train))
    data_train_neg=pd.concat([data_train_neg,data_neg.iloc[idx]])
    data_train_neg=data_train_neg.drop_duplicates()
    
    idx_removed=data_train_neg.index
    data_neg=data_neg[~data_neg.index.isin(idx_removed)]
    data_neg.index=range(len(data_neg))
    
    data_test=pd.read_csv('CoPheeKSA/test/w_feature/test_'+str(i)+'.csv',index_col=0)
    data_test=data_test.drop_duplicates()
    data_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    #data_test=data_test[~data_test['PSSM'].isna()]
    
    idx=random.sample(range(len(data_test)),25)
    data_eval=data_test.iloc[idx,:]
    idx2=[i for i in range(len(data_test)) if i not in idx]
    data_test=data_test.iloc[idx2,:]
    
    idx=random.sample(range(len(data_neg)),10*len(data_eval))
    data_neg_eval=data_neg.iloc[idx,:]
    idx2=[i for i in range(len(data_neg)) if i not in idx]
    data_neg=data_neg.iloc[idx2,:]
    
    data_train=data_train[col_idx]
    data_eval=data_eval[col_idx]
    data_test=data_test[col_idx]
    data_neg=data_neg[col_idx]
    data_train_neg=data_train_neg[col_idx]
    data_neg_eval=data_neg_eval[col_idx]
    
    X_train=pd.concat([data_train,data_train_neg])
    y_train=np.concatenate((np.repeat(1,len(data_train)),np.repeat(0,len(data_train_neg))))
    X_train=X_train[col_idx]
    X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    
    X_eval=pd.concat([data_eval,data_neg_eval])
    y_eval=np.concatenate((np.repeat(1,len(data_eval)),np.repeat(0,len(data_neg_eval))))
    X_eval=X_eval[col_idx]
    X_eval.replace([np.inf, -np.inf], np.nan, inplace=True)
    deval = xgb.DMatrix(X_eval,label=y_eval)
    param = {'max_depth': 2, 'eta': 0.2, 'objective': 'binary:logistic','verbosity':0}
    param['nthread'] = 4
    param['eval_metric'] = 'auc'
    evallist = [(dtrain, 'train'),(deval, 'eval')]
    num_round = 300
    bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=10)
    
    idx=random.sample(range(len(data_neg)),10*len(data_test))
    data_neg2=data_neg.iloc[idx,:]
    X_test=pd.concat([data_test,data_neg2])
    X_test=X_test[col_idx]
    X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    y_test=np.concatenate((np.repeat(1,len(data_test)),np.repeat(0,len(data_neg2))))
    dtest = xgb.DMatrix(X_test, label=y_test)
    y_pred = bst.predict(dtest)

    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred)
    auc_tmp=metrics.auc(fpr, tpr)
    logging.info(auc_tmp)
    
    auc.append(auc_tmp)
    label.append('Dynamic_features')

In [ ]:
#Supervised model trained with dynamic features + CoPheeMap info + KMap info
Cancer=['BRCA','CCRCC','COAD','GBM','HCC','HNSCC','LSCC','LUAD','OV','PDAC','UCEC'] 
col_idx=['cophee_emb0', 'cophee_emb1', 'cophee_emb2', 'cophee_emb3', 'cophee_emb4', 
         'cophee_emb5', 'cophee_emb6', 'cophee_emb7', 'cophee_emb8', 'cophee_emb9', 
         'cophee_emb10','cophee_emb11', 'cophee_emb12', 'cophee_emb13', 'cophee_emb14', 
         'cophee_emb15', 'k_emb0', 'k_emb1', 'k_emb2', 'k_emb3','k_emb4', 'k_emb5', 
         'k_emb6', 'k_emb7', 'k_emb8', 'k_emb9', 'k_emb10',
         'k_emb11', 'k_emb12', 'k_emb13', 'k_emb14', 'k_emb15','BRCA_spc_k_s', 
         'CCRCC_spc_k_s', 'COAD_spc_k_s', 'GBM_spc_k_s',
         'HCC_spc_k_s', 'HNSCC_spc_k_s', 'LSCC_spc_k_s', 'LUAD_spc_k_s',
         'OV_spc_k_s', 'PDAC_spc_k_s', 'UCEC_spc_k_s', 'BRCA_spc_kactivity_s',
         'CCRCC_spc_kactivity_s', 'COAD_spc_kactivity_s', 'GBM_spc_kactivity_s',
         'HCC_spc_kactivity_s', 'HNSCC_spc_kactivity_s', 'LSCC_spc_kactivity_s',
         'LUAD_spc_kactivity_s', 'OV_spc_kactivity_s', 'PDAC_spc_kactivity_s',
         'UCEC_spc_kactivity_s']
for i in range(10):
    data_train=pd.read_csv('CoPheeKSA/train/w_feature/train_'+str(i)+'.csv',index_col=0)
    data_train=data_train.drop_duplicates()
    data_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    #Avoid overfitting caused by missing values
    data_neg=pd.read_csv('Supplementary_table/Table_S3_negative_KSA_w_features.tsv.zip', compression='zip', sep='\t')
    data_neg=data_neg.drop_duplicates()
    
    kinases=list(set(data_train.kinase))
    data_train_neg=pd.DataFrame()
    for k in kinases:
        tmp=data_train[data_train.kinase==k]
        tmp2=data_neg[data_neg.kinase==k]
        if len(tmp2) > 10*len(tmp):
            idx=random.sample(range(len(tmp2)),10*len(tmp))
            tmp2=tmp2.iloc[idx]
        
        c=tmp[['BRCA_spc_kactivity_s',
             'CCRCC_spc_kactivity_s', 'COAD_spc_kactivity_s', 'GBM_spc_kactivity_s',
             'HCC_spc_kactivity_s', 'HNSCC_spc_kactivity_s', 'LSCC_spc_kactivity_s',
             'LUAD_spc_kactivity_s', 'OV_spc_kactivity_s', 'PDAC_spc_kactivity_s',
             'UCEC_spc_kactivity_s']].isna().sum()
        for idx_col in ['BRCA_spc_kactivity_s',
             'CCRCC_spc_kactivity_s', 'COAD_spc_kactivity_s', 'GBM_spc_kactivity_s',
             'HCC_spc_kactivity_s', 'HNSCC_spc_kactivity_s', 'LSCC_spc_kactivity_s',
             'LUAD_spc_kactivity_s', 'OV_spc_kactivity_s', 'PDAC_spc_kactivity_s',
             'UCEC_spc_kactivity_s']:
            num=int(c.loc[idx_col]/len(tmp)*len(tmp2))
            idx=random.sample(tmp2.index.tolist(),num)
            
            for z in idx:
                tmp2.loc[z, idx_col]=np.nan
            
        data_train_neg=pd.concat([data_train_neg,tmp2])
    idx=random.sample(range(len(data_neg)),2*len(data_train))
    data_train_neg=pd.concat([data_train_neg,data_neg.iloc[idx]])
    data_train_neg=data_train_neg.drop_duplicates()
    
    idx_removed=data_train_neg.index
    data_neg=data_neg[~data_neg.index.isin(idx_removed)]
    data_neg.index=range(len(data_neg))
    
    data_test=pd.read_csv('CoPheeKSA/test/w_feature/test_'+str(i)+'.csv',index_col=0)
    data_test=data_test.drop_duplicates()
    data_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    #data_test=data_test[~data_test['PSSM'].isna()]
    
    idx=random.sample(range(len(data_test)),25)
    data_eval=data_test.iloc[idx,:]
    idx2=[i for i in range(len(data_test)) if i not in idx]
    data_test=data_test.iloc[idx2,:]
    
    idx=random.sample(range(len(data_neg)),10*len(data_eval))
    data_neg_eval=data_neg.iloc[idx,:]
    idx2=[i for i in range(len(data_neg)) if i not in idx]
    data_neg=data_neg.iloc[idx2,:]
    
    
    data_train=data_train[col_idx]
    data_eval=data_eval[col_idx]
    data_test=data_test[col_idx]
    data_neg=data_neg[col_idx]
    data_train_neg=data_train_neg[col_idx]
    data_neg_eval=data_neg_eval[col_idx]
    
    X_train=pd.concat([data_train,data_train_neg])
    y_train=np.concatenate((np.repeat(1,len(data_train)),np.repeat(0,len(data_train_neg))))
    X_train=X_train[col_idx]
    X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    #X_train=X_train.replace(np.nan,0)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    
    X_eval=pd.concat([data_eval,data_neg_eval])
    y_eval=np.concatenate((np.repeat(1,len(data_eval)),np.repeat(0,len(data_neg_eval))))
    X_eval=X_eval[col_idx]
    X_eval.replace([np.inf, -np.inf], np.nan, inplace=True)
    deval = xgb.DMatrix(X_eval,label=y_eval)
    param = {'max_depth': 2, 'eta': 0.2, 'objective': 'binary:logistic','verbosity':0}
    param['nthread'] = 4
    param['eval_metric'] = 'auc'
    evallist = [(dtrain, 'train'),(deval, 'eval')]
    num_round = 300
    bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=10)
    
    idx=random.sample(range(len(data_neg)),10*len(data_test))
    data_neg2=data_neg.iloc[idx,:]
    X_test=pd.concat([data_test,data_neg2])
    X_test=X_test[col_idx]
    X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    y_test=np.concatenate((np.repeat(1,len(data_test)),np.repeat(0,len(data_neg2))))
    dtest = xgb.DMatrix(X_test, label=y_test)
    y_pred = bst.predict(dtest)

    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred)
    auc_tmp=metrics.auc(fpr, tpr)
    logging.info(auc_tmp)
    
    auc.append(auc_tmp)
    label.append('Dynamic_features+CoPheeMap+KMap')

In [ ]:
auc_results={}
auc_results['label']=np.array(label)
auc_results['AUROC']=np.array(auc)
auc_results=pd.DataFrame(auc_results)

#auc_results.to_csv('CoPheeKSA/auc_results.csv')

In [ ]:
auc_results=pd.read_csv('CoPheeKSA/auc_results.csv')

In [ ]:
tmp=pd.concat([auc_results[auc_results['label']=='PSSM'],
               auc_results[auc_results['label']=='CoPheeMap+KMap'],
               auc_results[auc_results['label']=='PSSM+CoPheeMap+KMap'],
               auc_results[auc_results['label']=='Dynamic_features'],
               auc_results[auc_results['label']=='Dynamic_features+CoPheeMap+KMap'],
               auc_results[auc_results['label']=='CoPheeKSA']])

In [ ]:
set(auc_results.label)

In [ ]:
plt.figure(figsize=(8, 4))
x = "label"
y = "AUROC"
#hue = "label"
ax = sns.boxplot(data=tmp, x=x, y=y,palette=['royalblue','gold','darkseagreen','orangered','orange']+[sns.color_palette()[8]])
add_stat_annotation(ax, data=tmp, x=x, y=y,
                    box_pairs=[('PSSM', 'PSSM+CoPheeMap+KMap'),
                               ('Dynamic_features','Dynamic_features+CoPheeMap+KMap'),
                               ('PSSM+CoPheeMap+KMap','CoPheeKSA'),
                               ('Dynamic_features+CoPheeMap+KMap','CoPheeKSA')],
                    test='t-test_ind', text_format='star', loc='outside', verbose=2)
plt.xticks(rotation=30,size=14,ha='right')
plt.yticks(size=14)
#plt.title('PSSM zscores',size=14)
plt.xlabel('')
plt.ylabel('AUROC',size=14)
#plt.legend(['PSSM','CoPheeMap+KMap','PSSM+CoPheeMap+KMap','Dynamic features','Dynamic features+CoPheeMap+KMap',
#            'CoPheeKSA'],loc='upper left', bbox_to_anchor=(1.03, 1),fontsize=12)
plt.savefig('CoPheeKSA/comparision_auroc_KSA.jpeg',bbox_inches='tight',dpi=300,pad_inches=0.1,orientation='portrait')
plt.show()

In [ ]:
#Train a final model CoPheeKSA using Dynamic features + PSSM + CoPheeMap + KMap
Cancer=['BRCA','CCRCC','COAD','GBM','HCC','HNSCC','LSCC','LUAD','OV','PDAC','UCEC'] 
col_idx=['cophee_emb0', 'cophee_emb1', 'cophee_emb2', 'cophee_emb3', 'cophee_emb4', 
         'cophee_emb5', 'cophee_emb6', 'cophee_emb7', 'cophee_emb8', 'cophee_emb9', 
         'cophee_emb10','cophee_emb11', 'cophee_emb12', 'cophee_emb13', 'cophee_emb14', 
         'cophee_emb15', 'k_emb0', 'k_emb1', 'k_emb2', 'k_emb3','k_emb4', 'k_emb5', 
         'k_emb6', 'k_emb7', 'k_emb8', 'k_emb9', 'k_emb10',
         'k_emb11', 'k_emb12', 'k_emb13', 'k_emb14', 'k_emb15','BRCA_spc_k_s', 
         'CCRCC_spc_k_s', 'COAD_spc_k_s', 'GBM_spc_k_s',
         'HCC_spc_k_s', 'HNSCC_spc_k_s', 'LSCC_spc_k_s', 'LUAD_spc_k_s',
         'OV_spc_k_s', 'PDAC_spc_k_s', 'UCEC_spc_k_s', 'BRCA_spc_kactivity_s',
         'CCRCC_spc_kactivity_s', 'COAD_spc_kactivity_s', 'GBM_spc_kactivity_s',
         'HCC_spc_kactivity_s', 'HNSCC_spc_kactivity_s', 'LSCC_spc_kactivity_s',
         'LUAD_spc_kactivity_s', 'OV_spc_kactivity_s', 'PDAC_spc_kactivity_s',
         'UCEC_spc_kactivity_s','PSSM']
for i in random.sample(range(10),1):
    data_train=pd.read_csv('CoPheeKSA/train/w_feature/train_'+str(i)+'.csv',index_col=0)
    data_train=data_train.drop_duplicates()
    data_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    data_train=data_train[~data_train['PSSM'].isna()]
    
    data_neg=pd.read_csv('Supplementary_table/Table_S3_negative_KSA_w_features.tsv.zip', compression='zip', sep='\t')
    data_neg=data_neg.drop_duplicates()
    data_neg.index=range(len(data_neg))
    
    kinases=list(set(data_train.kinase))
    data_train_neg=pd.DataFrame()
    for k in kinases:
        tmp=data_train[data_train.kinase==k]
        tmp2=data_neg[data_neg.kinase==k]
        if len(tmp2) > 10*len(tmp):
            idx=random.sample(range(len(tmp2)),10*len(tmp))
            tmp2=tmp2.iloc[idx]
        
        c=tmp[['BRCA_spc_kactivity_s',
             'CCRCC_spc_kactivity_s', 'COAD_spc_kactivity_s', 'GBM_spc_kactivity_s',
             'HCC_spc_kactivity_s', 'HNSCC_spc_kactivity_s', 'LSCC_spc_kactivity_s',
             'LUAD_spc_kactivity_s', 'OV_spc_kactivity_s', 'PDAC_spc_kactivity_s',
             'UCEC_spc_kactivity_s']].isna().sum()
        for idx_col in ['BRCA_spc_kactivity_s',
             'CCRCC_spc_kactivity_s', 'COAD_spc_kactivity_s', 'GBM_spc_kactivity_s',
             'HCC_spc_kactivity_s', 'HNSCC_spc_kactivity_s', 'LSCC_spc_kactivity_s',
             'LUAD_spc_kactivity_s', 'OV_spc_kactivity_s', 'PDAC_spc_kactivity_s',
             'UCEC_spc_kactivity_s']:
            num=int(c.loc[idx_col]/len(tmp)*len(tmp2))
            idx=random.sample(tmp2.index.tolist(),num)
            
            for z in idx:
                tmp2.loc[z, idx_col]=np.nan
            
        data_train_neg=pd.concat([data_train_neg,tmp2])
    idx=random.sample(range(len(data_neg)),2*len(data_train))
    data_train_neg=pd.concat([data_train_neg,data_neg.iloc[idx]])
    data_train_neg=data_train_neg.drop_duplicates()
    
    idx_removed=data_train_neg.index
    data_neg=data_neg[~data_neg.index.isin(idx_removed)]
    data_neg.index=range(len(data_neg))
    
    data_test=pd.read_csv('CoPheeKSA/test/w_feature/test_'+str(i)+'.csv',index_col=0)
    data_test=data_test.drop_duplicates()
    data_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    data_test=data_test[~data_test['PSSM'].isna()]
    
    idx=random.sample(range(len(data_test)),25)
    data_eval=data_test.iloc[idx,:]
    idx2=[i for i in range(len(data_test)) if i not in idx]
    data_test=data_test.iloc[idx2,:]
    
    idx=random.sample(range(len(data_neg)),10*len(data_eval))
    data_eval_neg=data_neg.iloc[idx]
    idx_removed=data_eval_neg.index
    data_neg=data_neg[~data_neg.index.isin(idx_removed)]
    data_neg.index=range(len(data_neg))
    
    idx=random.sample(range(len(data_neg)),10*len(data_test))
    data_test_neg=data_neg.iloc[idx]
    idx_removed=data_test_neg.index
    data_neg=data_neg[~data_neg.index.isin(idx_removed)]
    
    data_train=data_train[col_idx]
    data_eval=data_eval[col_idx]
    data_test=data_test[col_idx]
    data_neg=data_neg[col_idx]
    data_train_neg=data_train_neg[col_idx]
    data_eval_neg=data_eval_neg[col_idx]
    
    X_train=pd.concat([data_train,data_train_neg])
    y_train=np.concatenate((np.repeat(1,len(data_train)),np.repeat(0,len(data_train_neg))))
    X_train=X_train[col_idx]
    X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    
    X_eval=pd.concat([data_eval,data_eval_neg])
    y_eval=np.concatenate((np.repeat(1,len(data_eval)),np.repeat(0,len(data_eval_neg))))
    X_eval=X_eval[col_idx]
    X_eval.replace([np.inf, -np.inf], np.nan, inplace=True)
    deval = xgb.DMatrix(X_eval,label=y_eval)
    param = {'max_depth': 2, 'eta': 0.2, 'objective': 'binary:logistic','verbosity':0}
    param['nthread'] = 4
    param['eval_metric'] = 'auc'
    evallist = [(dtrain, 'train'),(deval, 'eval')]
    num_round = 300
    bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=10)
    #bst.save_model('CoPheeKSA/model/best_xgboost.model')
    
    idx=random.sample(range(len(data_neg)),10*len(data_test))
    data_neg2=data_neg.iloc[idx,:]
    X_test=pd.concat([data_test,data_neg2])
    X_test=X_test[col_idx]
    X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    y_test=np.concatenate((np.repeat(1,len(data_test)),np.repeat(0,len(data_neg2))))
    dtest = xgb.DMatrix(X_test, label=y_test)
    y_pred = bst.predict(dtest)

    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred)
    auc_tmp=metrics.auc(fpr, tpr)
    logging.info(auc_tmp)

In [ ]:
#Store the test data
idx=random.sample(range(len(data_neg)),100*len(data_test))
data_neg2=data_neg.iloc[idx,:]
X_test=pd.concat([data_test,data_neg2])
X_test=X_test[col_idx]
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
y_test=np.concatenate((np.repeat(1,len(data_test)),np.repeat(0,len(data_neg2))))
dtest = xgb.DMatrix(X_test, label=y_test)
y_pred = bst.predict(dtest)
y_test=pd.DataFrame(y_test)

#X_test.to_csv('CoPheeKSA/model/X_test.csv')
#y_test.to_csv('CoPheeKSA/model/y_test.csv')

In [ ]:
#X_test=pd.read_csv('CoPheeKSA/model/X_test.csv',index_col=0)
#y_test=pd.read_csv('CoPheeKSA/model/y_test.csv',index_col=0)

In [ ]:
bst = xgb.Booster()
bst.load_model('CoPheeKSA/model/best_xgboost.model')

In [ ]:
#Predict KSAs using CoPheeKSA
#Warning: Long running time and original data will be replaced by the new data
bst = xgb.Booster()
bst.load_model('CoPheeKSA/model/best_xgboost.model')
mypath='CoPheeKSA/prediction_data/'
onlyfiles = [f for f in listdir(mypath) if f!='.DS_Store']
onlyfiles = [f for f in onlyfiles if isfile(''.join([mypath, f]))]

path2='CoPheeKSA/PSSM/ST/'
onlyfiles2 = [f for f in listdir(path2) if f!='.DS_Store']
onlyfiles2 = [f for f in onlyfiles2 if 'family' not in f]
ST_k=[f.split('_')[0] for f in onlyfiles2]
col_idx=['cophee_emb0', 'cophee_emb1', 'cophee_emb2', 'cophee_emb3', 'cophee_emb4', 
         'cophee_emb5', 'cophee_emb6', 'cophee_emb7', 'cophee_emb8', 'cophee_emb9', 
         'cophee_emb10','cophee_emb11', 'cophee_emb12', 'cophee_emb13', 'cophee_emb14', 
         'cophee_emb15', 'k_emb0', 'k_emb1', 'k_emb2', 'k_emb3','k_emb4', 'k_emb5', 
         'k_emb6', 'k_emb7', 'k_emb8', 'k_emb9', 'k_emb10',
         'k_emb11', 'k_emb12', 'k_emb13', 'k_emb14', 'k_emb15','BRCA_spc_k_s', 
         'CCRCC_spc_k_s', 'COAD_spc_k_s', 'GBM_spc_k_s',
         'HCC_spc_k_s', 'HNSCC_spc_k_s', 'LSCC_spc_k_s', 'LUAD_spc_k_s',
         'OV_spc_k_s', 'PDAC_spc_k_s', 'UCEC_spc_k_s', 'BRCA_spc_kactivity_s',
         'CCRCC_spc_kactivity_s', 'COAD_spc_kactivity_s', 'GBM_spc_kactivity_s',
         'HCC_spc_kactivity_s', 'HNSCC_spc_kactivity_s', 'LSCC_spc_kactivity_s',
         'LUAD_spc_kactivity_s', 'OV_spc_kactivity_s', 'PDAC_spc_kactivity_s',
         'UCEC_spc_kactivity_s','PSSM']
for f in onlyfiles:
    kinase=f.split('.')[0]
    if kinase in ST_k:
        logging.info(kinase)
        data=pd.read_csv(''.join([mypath,f]),index_col=0)
        data.replace([np.inf, -np.inf], np.nan, inplace=True)
        data.index=data.sites
        score=[]
        data=data[col_idx]
        y_test=np.repeat(0,len(data))
        dtest = xgb.DMatrix(data, label=y_test)
        prediction = bst.predict(dtest)
        data['prediction']=np.array(prediction)
        data.to_csv(''.join(['CoPheeKSA/prediction/',f]))

In [ ]:
#Threshold when LLR>0.55
thred_55=0.767626

#Collect positive predicted KSAs
mypath='CoPheeKSA/prediction/'
onlyfiles = [f for f in listdir(mypath) if f!='.DS_Store']
ks1=[i.split('_')[0] for i in onlyfiles]
logging.info(len(ks1))
combined_pos=pd.DataFrame()
for f in onlyfiles:
    logging.info(f)
    kinase=f.split('.')[0]
    data=pd.read_csv(''.join([mypath,'/',f]),index_col=0)
    data=data.sort_values('prediction',ascending=False)
    data_pos=data[data.prediction>thred_55]
    #data_pos=data_pos.iloc[:1000]
    data_pos['kinase']=np.repeat(f.split('.')[0],len(data_pos))
    logging.info(len(data_pos))
    combined_pos=pd.concat([combined_pos,data_pos])

In [ ]:
#combined_pos.to_csv('CoPheeKSA/results/K_S_CoPhee_llr55_w_features.csv')